In [28]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [29]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [30]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [31]:

course_id = course_ids.get('청도')
course_id

'MGC002'

In [32]:
folders_candidate =glob.glob(os.path.join(out_folder,course_id,'*'))
folders_candidate

['D:\\ToAWS2\\MGC002\\20230907']

In [33]:
target_folder = folders_candidate[0]
target_folder

'D:\\ToAWS2\\MGC002\\20230907'

In [34]:
target_date = os.path.split(target_folder)[-1]
target_date 

'20230907'

Gather all data json files from Output Folder

In [35]:
dataJsonList = glob.glob(os.path.join(out_folder,course_id,target_date,'**/data*.json'), recursive=True)
dataJsonList

['D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_레이크1H\\data20230907청도그레이스_레이크1H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_레이크2H\\data20230907청도그레이스_레이크2H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_레이크4H\\data20230907청도그레이스_레이크4H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴1H\\data20230907청도그레이스_마운틴1H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴2H\\data20230907청도그레이스_마운틴2H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴3H\\data20230907청도그레이스_마운틴3H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴4H\\data20230907청도그레이스_마운틴4H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴5H\\data20230907청도그레이스_마운틴5H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴6H\\data20230907청도그레이스_마운틴6H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴7H\\data20230907청도그레이스_마운틴7H.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_마운틴너스리포장\\data20230907청도그레이스_마운틴너스리포장.json',
 'D:\\ToAWS2\\MGC002\\20230907\\청도그레이스_스타트퍼팅그린\\data20230907청도그레이스_스타트퍼팅그린.json']

In [36]:
folderinDataJson = []
totalPhotoJson = []

for dataJson_ in dataJsonList:
  with open(dataJson_, "r") as _json:
    dataJson= json.load(_json)


  for data_ in dataJson:
    # if data_['dest']['thumb'][0] !="M" :
      data_['dest']['thumb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'thumb','thumbrgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['rgb'][0] !="M" :
      data_['dest']['rgb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'rgb','rgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['ndvi'][0] !="M" :
      data_['dest']['ndvi'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'ndvi','ndvi{}.JPG'.format( data_['id']) )
    
  
  totalPhotoJson.extend(dataJson)

  for Json_ in dataJson:
    if Json_['destFolder'] not in folderinDataJson:
      folderinDataJson.append(Json_['destFolder'])




In [37]:
len(totalPhotoJson)

9891

In [38]:
totalPhotoJson[0]

{'id': '20230907150744_1286447571944_356629456389_156910000',
 'originalFileJPG': 'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_D.JPG',
 'originalFileGrp': ['NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_MS_G.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_MS_NIR.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\\2023_0907_청도그레이스_레이크1H_0.5m\\20230907_1507-1513_패취류,섬머패취,켄터키,티,잡초,이종잔디,한국잔디\\DJI_20230907150744_0001_MS_R.TIF',
  'NIA_2023_52_잔디\\10_원시데이터(3사_분류대상)\\2023_0907_청도그레이스_맑은날_구름조금_데이터 1차 분류_완\\청도그레이스_레이크1H\

In [39]:
save_name = os.path.join(out_folder,course_id,target_date, 'photo.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(totalPhotoJson, final , ensure_ascii=False)

In [40]:
filestructure = []


for folder_ in folderinDataJson:
  fileinfo = {}
  fileinfo["path"] = folder_
  fileinfo["file_cnt"] = len([ x for x in totalPhotoJson if x["destFolder"] == folder_])
  fileinfo["labeled_file_cnt"] = 0
  filestructure.append(fileinfo)
  

In [41]:
save_name = os.path.join(out_folder,course_id,target_date, 'filepath_'+course_id + '_'+target_date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(filestructure, final , ensure_ascii=False)